# <span style="font-size:26px">Enhancing energy transfer using Edge Plasmon Polarization

## <span style="font-size:23px">1.Computational Method
### <span style="font-size:20px">In this study, three-dimensional finite-difference time-domain (FDTD) simulations were performed using the Meep package to numerically investigate resonance energy transfer in the vicinity of a metallic edge.

## 
<span style="font-size:23px">2.Principle
### <span style="font-size:20px">2.1.
### <span style="font-size:20px">According to the generalized theory of resonance energy transfer (RET) developed by Wu *et al.* (J. Phys. Chem. Lett. **9**, 7032 (2018)), the rate of energy transfer between a donor and an acceptor in an arbitrary electromagnetic environment can be written as
$$
W_{\mathrm{ET}}
=\frac{9 c^{4}W_{r}^{D}}{8 \pi}
\int_{0}^{\infty}
\mathrm{d}\omega\,
\frac{\sigma(\omega)\, I(\omega)}{\omega^{4}}\,
F(\mathbf{r}_{D}, \mathbf{r}_{A}, \omega)
$$
### <span style="font-size:20px"> where $𝑐$ is the speed of light in vacuum, $W_{r}^{D}$ denotes the total radiative decay rate of the donor in free space, and $𝜔$ is the angular frequency.Here, $𝜎(𝜔)$represents the absorption cross section of the acceptor, and $𝐼(𝜔)$is the normalized emission spectrum of the donor, satisfying$$\int_{0}^{\infty} d\omega \, I(\omega) = 1 .$$
### <span style="font-size:20px">The factor $F(\mathbf{r}_D, \mathbf{r}_A, \omega)$ is the electromagnetic coupling factor, which accounts for the interaction between the donor located at $\mathbf{r}_D$ and the acceptor at $\mathbf{r}_A$. This factor incorporates both radiative and nonradiative contributions mediated by the surrounding electromagnetic environment.

### 2.2
### Assuming that the absorption spectrum of the acceptor is concentrated at a single angular frequency $\omega_0$, the absorption cross section is approximated as
### $$\sigma(\omega) = \sigma_0 \, \delta(\omega - \omega_0)$$
### When the single-frequency approximation is adopted, the frequency distribution is assumed to be concentrated at the central angular frequency $\omega_0$. Consequently, it can be represented by a Dirac delta function as
### $$\delta(\omega - \omega_0)$$
### Therefore, under the single-frequency approximation, the RET rate can be simplified to
###  
$$
W_{\mathrm{ET}} \approx \frac{9 c^{4} W_{r}^{D}}{8\pi}\,
\frac{\sigma_{0}\, I(\omega_{0})}{\omega_{0}^{4}}\,
F(\mathbf{r}_{D}, \mathbf{r}_{A}, \omega_{0})
$$



### 2.3
### When the orientation of the acceptor dipole $\hat{\mathbf{n}}_A$ and the angular frequency $\omega_0$ are fixed, and the donor transition dipole amplitude is $p_{\mathrm{ex}} = 1$, the electromagnetic coupling factor can be simplified accordingly.
### Therefore, the enhancement factor $\gamma$ of resonance energy transfer is defined as
### 
$$
\gamma \equiv \frac{W_{\mathrm{ET}}}{W_{\mathrm{ET},0}} $$
### Eliminating the constant term can be written as
### 
$$
\gamma \approx 
\frac{F(\mathbf{r}_{D}, \mathbf{r}_{A}, \omega_{0})}
     {F_{0}(\mathbf{r}_{D}, \mathbf{r}_{A}, \omega_{0})}
=
\frac{\left|\hat{\mathbf{n}}_A \cdot \mathbf{E}_D(\mathbf{r}_A,\omega_0)\right|^2}
     {\left|\hat{\mathbf{n}}_A \cdot \mathbf{E}_D^{(0)}(\mathbf{r}_A,\omega_0)\right|^2}
$$





## <span style="font-size:23px">3.Model Configuration
### <span style="font-size:20px">A continuous-wave, x-polarized dipole source was placed in free space, and the frequency-domain electric field distribution was sampled along the +y direction from the source position.
### <span style="font-size:20px">To introduce an edge geometry, a finite-thickness metallic film with a thickness of 40 nm was included in the left half-space (x < 0), forming a sharp metallic edge at x = 0.

## <span style="font-size:23px">4.Evaluation
### <span style="font-size:20px">The distance-dependent field enhancement was quantified by comparing the squared electric field amplitude |Ex|² in the presence of the metal with that in free space. This enhancement factor was used to evaluate the influence of the metallic edge on the resonance energy transfer behavior.


## <span style="font-size:23px">Notice

### <span style="font-size:20px">Rename the file to energy_transfer_40nm_xx_1.25.py
### <span style="font-size:20px">The simulation was performed on a remote computing server, accessed via an SSH connection using MobaXterm.
### <span style="font-size:20px">Execute the following command: python energy_transfer_40nm_xx_1.25.py --cases air,au,pec --nproc 3 --outdir out --time 30


In [ ]:
import os
# Limit the number of CPU threads used by BLAS/OMP libraries.
# This helps avoid oversubscription (too many threads) when using multiprocessing.
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

import argparse
import numpy as np
import meep as mp
import meep.materials as mat
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from concurrent.futures import ProcessPoolExecutor, as_completed


# =========================
# Global parameters
# =========================
TAG = "3d_RET_Ex_Rscan_y_metal_t40nm_fcen1.25_edge"  # used to name output files

# Single-frequency excitation (units: 1/um). lambda0 = 1/f0 (um).
f0 = 1.25
lambda0 = 1.0 / f0

# Simulation numerics
resolution = 100     # pixels/um  (≈10 nm per voxel)
T_run = 30.0         # run time (time units in Meep)
pml_thick = 0.5      # PML thickness (um)
margin = 0.02        # extra padding from PML (um)

# Simulation cell size (um)
sx, sy, sz = 1.5, 7.5, 1.5

# Source position (um)
SRC_X = 0.01
SRC_Y = -2.85
SRC_Z = 0.0

# Sample distances R along +y from the source: 0.01*lambda0 to 10*lambda0 (log-spaced)
R_min_lam, R_max_lam, NUM_R = 1e-2, 10.0, 6000
R_list = lambda0 * np.geomspace(R_min_lam, R_max_lam, NUM_R)

# Metal film thickness along z (um); film occupies z in [-METAL_T_UM, 0]
METAL_T_UM = 0.04


# =========================
# Interpolation helpers
# =========================
def interp_loglog(Xsrc, Ysrc, Xdst, tiny=1e-30):
    # Log-log interpolation for positive-valued curves (here used for |Ex|^2).
    # Clamp Y to tiny to avoid log(0).
    ylog = np.log(np.maximum(Ysrc, tiny))
    return np.exp(np.interp(np.log(Xdst), np.log(Xsrc), ylog))


def interp_complex(Xsrc, Ysrc, Xdst):
    # Linear interpolation for complex arrays:
    # interpolate real/imag parts separately, then recombine.
    re = np.interp(Xdst, Xsrc, np.real(Ysrc))
    im = np.interp(Xdst, Xsrc, np.imag(Ysrc))
    return re + 1j * im


# =========================
# Run one simulation case
# =========================
def run_case(which: str, T=T_run):
    # Runs a single case: 'air', 'au', or 'pec'.
    # Returns:
    #   R_safe: distances (um) along +y that stay away from PML
    #   Ex/Ey/Ez: complex frequency-domain fields sampled along the line
    #   Ex2_interp: |Ex|^2 on the same R grid (only used internally for gamma)
    cell = mp.Vector3(sx, sy, sz)

    # PML absorbing boundaries on all sides
    pmls = [
        mp.PML(thickness=pml_thick, direction=mp.X),
        mp.PML(thickness=pml_thick, direction=mp.Y),
        mp.PML(thickness=pml_thick, direction=mp.Z),
    ]

    # Geometry: for Au/PEC, add a metal slab in the left half (x in [-0.75, 0])
    # with thickness METAL_T_UM in z, and top surface at z=0.
    geometry = []
    if which.lower() in ("au", "pec"):
        material = mat.Au if which.lower() == "au" else mp.metal
        geometry.append(
            mp.Block(
                size=mp.Vector3(0.75, mp.inf, METAL_T_UM),
                center=mp.Vector3(-0.75 / 2.0, 0.0, -0.5 * METAL_T_UM),
                material=material,
            )
        )

    # Continuous-wave source polarized along x (Ex)
    sources = [
        mp.Source(
            mp.ContinuousSource(frequency=f0),
            component=mp.Ex,
            center=mp.Vector3(SRC_X, SRC_Y, SRC_Z),
        )
    ]

    # Build simulation
    sim = mp.Simulation(
        cell_size=cell,
        geometry=geometry,
        boundary_layers=pmls,
        resolution=resolution,
        dimensions=3,
        default_material=mp.air,
        sources=sources,
        symmetries=[],
        filename_prefix=f"{TAG}_{which}",
    )

    # Desired y sampling positions = SRC_Y + R_list
    y_targets = SRC_Y + R_list

    # Keep only points that are inside the "safe" region (away from PML)
    y_left = -sy / 2.0 + pml_thick + margin
    y_right = sy / 2.0 - pml_thick - margin
    mask = (y_targets >= y_left) & (y_targets <= y_right)
    y_safe_targets = y_targets[mask]

    # If too few points survive, the line is too short: increase sy or reduce max R
    if len(y_safe_targets) < 50:
        raise RuntimeError(f"[{which}] DFT line too short; increase sy or reduce R_max_lam.")

    # The actual DFT monitor runs from y0 to y1 (continuous line segment)
    y0, y1 = float(y_safe_targets.min()), float(y_safe_targets.max())

    # Add a frequency-domain (DFT) field monitor along a 1D line (x fixed, z fixed, varying y)
    # We request Ex, Ey, Ez at a single frequency f0.
    dft = sim.add_dft_fields(
        [mp.Ex, mp.Ey, mp.Ez],
        f0, 0, 1,
        where=mp.Volume(
            center=mp.Vector3(SRC_X, 0.5 * (y0 + y1), SRC_Z),
            size=mp.Vector3(0.0, (y1 - y0), 0.0),
        ),
    )

    # Run time-domain simulation until steady-state is reached (or long enough)
    sim.run(until=T)

    # Extract complex frequency-domain fields on the DFT line grid
    Ex = sim.get_dft_array(dft, mp.Ex, 0)
    Ey = sim.get_dft_array(dft, mp.Ey, 0)
    Ez = sim.get_dft_array(dft, mp.Ez, 0)

    # Native y-grid of the DFT line (same length as the returned arrays)
    ys = np.linspace(y0, y1, len(Ex))

    # Convert to distance R from the source along +y (um)
    R_native = ys - SRC_Y

    # Compute |Ex|^2 on the native grid (used only for gamma, not saved)
    Ex2_line = np.abs(Ex) ** 2

    # Reset Meep state (frees memory for multiprocessing runs)
    sim.reset_meep()

    # Target R grid (safe sampled R points)
    R_safe = y_safe_targets - SRC_Y

    # Interpolate |Ex|^2 in log-log space (good for wide dynamic range / power-law-like behavior)
    Ex2_interp = interp_loglog(R_native, Ex2_line, R_safe)

    # Interpolate complex Ex/Ey/Ez to the same R_safe grid (linear in real/imag)
    Ex_safe = interp_complex(R_native, Ex, R_safe)
    Ey_safe = interp_complex(R_native, Ey, R_safe)
    Ez_safe = interp_complex(R_native, Ez, R_safe)

    return which, R_safe, Ex2_interp, Ex_safe, Ey_safe, Ez_safe


# =========================
# Saving & plotting
# =========================
def save_curve(which, R, Ex, Ey, Ez, outdir="out"):
    # Save only the complex fields and coordinate R.
    # (|Ex|^2 can always be reconstructed later from Ex.)
    os.makedirs(outdir, exist_ok=True)
    np.savez(
        os.path.join(outdir, f"{TAG}_{which}_R_Ex.npz"),
        which=which,
        R=R,
        Ex=Ex, Ey=Ey, Ez=Ez,
        f0=f0, lambda0=lambda0,
        SRC_X=SRC_X, SRC_Y=SRC_Y, SRC_Z=SRC_Z,
        resolution=resolution, T_run=T_run,
        sx=sx, sy=sy, sz=sz, pml_thick=pml_thick,
        metal_t=METAL_T_UM,
    )


def plot_gamma(curves, outdir="out"):
    # curves stores (R, |Ex|^2) for each case, used to compute enhancement:
    # gamma_Ex(R) = |Ex_mat(R)|^2 / |Ex_air(R)|^2
    if "air" not in curves:
        raise RuntimeError("Need 'air' case as denominator.")

    R_air, Ex2_air = curves["air"]
    Rmin = R_air.min()
    Rmax = R_air.max()

    plt.figure(figsize=(5, 5))
    legend_items = []

    # Compute gamma only for materials relative to air
    for mat_name in ("au", "pec"):
        if mat_name in curves:
            R_m, Ex2_m = curves[mat_name]

            # Build a common R grid where both air and material data exist
            R0 = max(Rmin, R_m.min())
            R1 = min(Rmax, R_m.max())
            R_common = np.geomspace(R0, R1, 600)

            # Interpolate |Ex|^2 onto the common grid (log-log)
            G_air = interp_loglog(R_air, Ex2_air, R_common)
            G_mat = interp_loglog(R_m, Ex2_m, R_common)

            # Enhancement (avoid divide-by-zero)
            gamma = G_mat / np.maximum(G_air, 1e-30)

            # Plot using normalized distance R/lambda0 on a log x-axis
            R_over_lambda = R_common / lambda0
            label = f"{mat_name.upper()} (t=40 nm)"
            plt.semilogx(R_over_lambda, gamma, "-", lw=2.0, label=label)
            legend_items.append(mat_name)

            # Save CSV for later analysis
            os.makedirs(outdir, exist_ok=True)
            np.savetxt(
                os.path.join(outdir, f"{TAG}_gammaEx_vs_R_{mat_name}.csv"),
                np.column_stack([R_common, R_over_lambda, gamma]),
                delimiter=",",
                header="R(um),R/lambda0,gamma_Ex",
                comments="",
            )

    # Figure formatting
    plt.xlim(R_min_lam, R_max_lam)
    plt.xlabel(r"$R/\lambda_0$")
    plt.ylabel(r"Enhancement $\gamma_{Ex}$")

    ax = plt.gca()
    ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=False))
    ax.ticklabel_format(style="plain", axis="y")

    plt.grid(True, which="both", ls="--", alpha=0.5)
    if legend_items:
        plt.legend()

    plt.tight_layout()
    figpath = os.path.join(outdir, f"{TAG}_gammaEx_vs_R.png")
    plt.savefig(figpath, dpi=300)
    plt.close()
    print(f"[saved] {figpath}")


# =========================
# CLI entry point
# =========================
def main():
    # Parse command-line options
    parser = argparse.ArgumentParser(
        description="RET Ex along +y; R in [0.01, 10] lambda0; 40 nm film with an edge at x=0"
    )
    parser.add_argument("--cases", type=str, default="air,au,pec",
                        help="comma-separated cases: air,au,pec")
    parser.add_argument("--nproc", type=int, default=3,
                        help="number of worker processes")
    parser.add_argument("--outdir", type=str, default="out",
                        help="output directory for npz/csv/png")
    parser.add_argument("--time", type=float, default=T_run,
                        help=f"simulation time (default {T_run})")
    args = parser.parse_args()

    which_list = [w.strip().lower() for w in args.cases.split(",") if w.strip()]
    valid = {"air", "au", "pec"}
    for w in which_list:
        if w not in valid:
            raise ValueError(f"--cases contains unsupported option: {w}; valid are {sorted(valid)}")

    # Print run summary
    print(f"[{TAG}] f0={f0:.3f} (lambda0={lambda0:.4f} um), res={resolution}, T={args.time}")
    print(f"cell=({sx},{sy},{sz}) um, PML={pml_thick} um; src @ (x={SRC_X}, y={SRC_Y}, z={SRC_Z} um)")
    print(
        f"scan: R from {R_min_lam} to {R_max_lam} lambda0 "
        f"({R_list.min():.4g}–{R_list.max():.4g} um), metal t={METAL_T_UM*1000:.0f} nm"
    )
    print(f"cases={which_list}, nproc={args.nproc}, outdir={args.outdir}")

    # curves holds only what is needed to compute gamma: (R, |Ex|^2)
    curves = {}

    # Run cases in parallel
    with ProcessPoolExecutor(max_workers=min(args.nproc, len(which_list))) as ex:
        fut2which = {ex.submit(run_case, w, args.time): w for w in which_list}

        for fut in as_completed(fut2which):
            w = fut2which[fut]
            try:
                which_out, R_out, Ex2_out, Ex_out, Ey_out, Ez_out = fut.result()

                # Store |Ex|^2 for gamma computation (not written to disk)
                curves[which_out] = (R_out, Ex2_out)

                # Save complex fields to disk
                save_curve(which_out, R_out, Ex_out, Ey_out, Ez_out, outdir=args.outdir)

                print(f"[done] {which_out}: R in [{R_out.min():.4g}, {R_out.max():.4g}] um, N={len(R_out)}")
            except Exception as e:
                print(f"[ERROR] case={w}: {e}")

    # Compute and plot enhancement ratios (requires air)
    if "air" in curves:
        plot_gamma(curves, outdir=args.outdir)
    else:
        print("[skip plot] no 'air' curve; cannot compute enhancement ratios.")


if __name__ == "__main__":
    main()
